In [1]:
import pyodbc
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po
model = pe.ConcreteModel("Cargacamiones")
import random
import sys
random.seed(0)

## Preprocesado de los datos

In [2]:
path_demanda=r"C:\Users\jmormal\PycharmProjects\pythonProject3\Demanda_validacion.xlsx"

In [3]:
df_demanda=pd.read_excel(path_demanda)

In [4]:
df_demanda=df_demanda.groupby("Collection in supplier").sum().iloc[: , :2]
df_demanda
df_demanda

,Magnum,A-cont
Collection in supplier,,
day 1,138,16
day 2,171,48
day 3,204,20
day 4,123,24
day 5,198,28
day 6,123,24
day 7,123,24
day 8,198,32
day 9,126,20


In [5]:
l=[]
for t in range(1,10):
    l1=[1,t,df_demanda.iloc[t-1 , 0 ]]
    l2=[2,t,df_demanda.iloc[t-1 , 1 ]]
    l.append(l1)
    l.append(l2)
df=pd.DataFrame(l,columns=["Producto","t","Demanda"])

In [6]:
path=r"C:\Users\jmormal\PycharmProjects\pythonProject3\Transporte_cubicaje_validacion.accdb"
conn = pyodbc.connect('Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+path+';')
cursor = conn.cursor()

Familias Carga de datos

In [7]:
cursor.execute('select IdProducto from Productos')
productos=[]
for row in cursor.fetchall():
    productos.append(row[0])

In [8]:
cursor.execute('select IdFamilia from Familias')
familias=[]
for row in cursor.fetchall():
    familias.append(row[0])

In [9]:
cursor.execute('select IdCamion from Camiones WHERE IdCamion<4')
camiones=[]
for i in range(1,7):
    camiones.append(i)

In [10]:
cursor.execute('select IdDia from Dias WHERE "IdDia"<=19')
dias=[]
for row in cursor.fetchall():
    dias.append(row[0])
        

In [11]:
cursor.execute('select IdDia from Dias')
dias1=[]
for row in cursor.fetchall():
    dias1.append(row[0])



In [12]:
columnas=[1,2]
A1=[6,7]
B1=[4,2]
productos

[2, 1]

## Familias Pyomo

In [13]:
model.i = pe.Set(initialize=productos)
model.f = pe.Set(initialize=familias)
model.c = pe.Set(initialize=camiones)
model.t = pe.Set(initialize=dias)
model.t1 = pe.Set(initialize=dias1)
model.col=pe.Set(initialize=columnas)
model.A1=pe.Set(initialize=A1)
model.B1=pe.Set(initialize=B1)

## Carga de datos productos

In [14]:
cursor.execute('select IdFamilia, LotePedido from Familias')
lotepedido={}
for row in cursor.fetchall():
    lotepedido.update({row[0]:row[1]})


In [15]:
cursor.execute('select IdFamilia, LotePedido from Familias')
CosteStock={}
for row in cursor.fetchall():
    CosteStock.update({row[0]:random.randint(7, 10)*0.001})


In [16]:
cursor.execute('select IdProducto , PiezasCont from Productos')
piezascont={}
for row in cursor.fetchall():
    piezascont.update({row[0]:row[1]})

In [17]:
cursor.execute('select IdProducto , LargoContedor from Productos')
piezas_largo_box={}
for row in cursor.fetchall():
    piezas_largo_box.update({row[0]:row[1]})

In [18]:
cursor.execute('select IdProducto , IdFamilia, beta from beta')
b={}
for row in cursor.fetchall():
    b.update({(row[0],row[1]):row[2]})


In [19]:
cursor.execute('select  IdProducto , IdDia, Demanda from Demanda')
demanda={}
for row in cursor.fetchall():
    # if row[0]>=13:
    #     demanda.update({(row[0],row[1]):row[2]})
    # else:
        demanda.update({(row[0],row[1]):row[2]})    

In [20]:
# cursor.execute('select IdProducto , StockInicial from Productos')
stockInicial={}
# for row in cursor.fetchall():
#     stockInicial.update({row[0]:row[1]})
cursor.execute('select  IdProducto , Demanda from Demanda where IdDia=1')

for row in cursor.fetchall():
    stockInicial.update({row[0]:row[1]*0})


In [21]:
model.i.__dict__["_init_values"]

In [22]:
cursor.execute('select  IdProducto , Valor from Productos')
valor={}
for row in cursor.fetchall():
    valor.update({row[0]:row[1]})

In [23]:
PC={}
for camion in camiones:
    PC.update({camion:1000})

In [24]:
L=78
nu=26/30
NT_lw=5
NT_up=10

In [25]:
model.LotePedido = pe.Param(model.f , initialize=lotepedido)
model.PiezasCont= pe.Param(model.i ,initialize=piezascont)
model.b= pe.Param(model.i, model.f ,initialize=b, default=0)
model.StockInicial= pe.Param(model.i, initialize=stockInicial)
model.demanda= pe.Param(model.i, model.t1,  initialize=demanda)
model.valor=pe.Param(model.i, initialize=stockInicial)
model.PC=pe.Param(model.c, initialize=PC)
model.L=pe.Param(initialize=L)
model.nu=pe.Param(initialize=nu)
model.NT_lw=pe.Param(initialize=NT_lw)
model.NT_up=pe.Param(initialize=NT_up)
model.LargoCont=pe.Param(model.i,initialize=piezas_largo_box)
model.CS=pe.Param(model.i, initialize=CosteStock)

# # Varibles

In [26]:
model.CantidadPedir = pe.Var(model.i, model.c, model.t,model.col, domain=pe.Integers)

In [27]:
model.CantidadPedir2 = pe.Var(model.i,model.f, model.c, model.t,model.col, domain=pe.Integers, bounds=(0,None) )
model.ContenedoresCamion=pe.Var(model.c,model.t,model.col,model.i, domain=pe.Integers)

In [28]:
model.Stock=pe.Var(model.i,model.t, bounds=(0,None))

In [29]:
model.RetrasoDemanda=pe.Var(model.i,model.t, bounds=(0,None))

In [30]:
model.K=pe.Var(model.f,model.c,model.t,model.col ,domain=pe.Integers)

In [31]:
model.Y=pe.Var(model.c,model.t ,domain=pe.Binary, initialize=0)

In [32]:
model.A=pe.Var(model.c,model.col,model.t, domain=pe.Integers, bounds=(0,4.5))
model.B=pe.Var(model.c,model.col, model.t, domain=pe.Integers, bounds=(0,12.5))
# model.A=pe.Var(model.c,model.col,model.t, domain=pe.Integers, bounds=(0,13))
# model.B=pe.Var(model.c,model.col, model.t, domain=pe.Integers, bounds=(0,13))


## Función Objetivo
Minimizar el número de camiones usado
$$\sum_{c,t}PC(c)Y(c,t)+ \sum CS(i)Stock(i,t)$$

In [33]:
expr = sum(model.PC[c]*model.Y[c,t] +sum(model.CS[i]*model.Stock[i,t] for i in model.i) for c in model.c for t in model.t)

In [34]:
 #Coste Transporte
model.objective = pe.Objective(sense=pe.minimize, expr=expr)

## Funciones auxiliares

In [35]:
def UsoCamion(model):
    return sum(model.L*model.Y[c,t] for c in model.c for t in model.t)-\
        sum(model.ContenedoresCamion[c,t] for c in model.c for t in model.t)

In [36]:
def CTransporte(model):
    return sum(model.PC[c]*model.Y[c,t] for c in model.c for t in model.t)

In [37]:
def NumeroCamiones(model):
    return sum(model.Y[c,t] for c in model.c for t in model.t)

## Restricciones

In [38]:
model.stocks=pe.ConstraintList()

$$Stock(i,t)=Stock(i,t-1)+\sum_{c,col}CantidadPedir(i,c,t,col)PiezasCont[i] -Demanda(i,t)$$

In [39]:
for i in model.i:
        lhs=model.Stock[i,1]
        rhs=model.StockInicial[i]-model.demanda[i,1]+sum(model.CantidadPedir[i,c,1,col]*model.PiezasCont[i] for c in model.c for col in model.col )
        model.stocks.add(lhs==rhs)

In [40]:
for t in list(model.t)[1:]:
    for i in model.i:
            lhs=model.Stock[i,t]
            rhs=model.Stock[i,t-1]-model.demanda[i,t]+sum(model.CantidadPedir[i,c,t,col]*model.PiezasCont[i] for c in model.c for col in model.col)
            model.stocks.add(lhs==rhs)

In [41]:
model.RcantidadPedir=pe.ConstraintList()

$$CantidadPedir2_{i,f,c,t,col}=K(f,c,t,col)*LotePedido(f)*b(i,f)$$

In [42]:
for i in model.i:
    for f in model.f:
        for c in model.c:
            for t in model.t:
                for col in model.col:
                    lhs=model.CantidadPedir2[i,f,c,t,col]
                    rhs=model.K[f,c,t,col]*model.LotePedido[f]*model.b[i,f]
                    model.RcantidadPedir.add(lhs==rhs)

In [43]:
model.RcantidadPedir2=pe.ConstraintList()

$$CantidadPedir_{i,c,t,col}= \sum_f CanitdadPedir2(i,f,c,t,col)$$

In [44]:
for i in model.i:
    for c in model.c:
        for t in model.t:
            for col in model.col:
                lhs=model.CantidadPedir[i,c,t,col]
                rhs=sum(model.CantidadPedir2[i,f,c,t,col] for f in model.f)
                model.RcantidadPedir2.add(lhs==rhs)

In [45]:
model.RDimensionesCamion=pe.ConstraintList()

$$ 1600A_{c,col,t} +1000B_{c,col,t}   \leq 13600* Y[c,t]$$

In [46]:
for c in model.c:
    for t in model.t:
        for col in model.col:
            lhs=13600*model.Y[c,t]
            rhs=(model.A[c,col,t]*1600+model.B[c,col,t]*1000)
            model.RDimensionesCamion.add(lhs>=rhs)

In [47]:
model.RCajasTotales=pe.ConstraintList()

$$ \forall c \in C \quad \forall t \in T \quad \sum_i CantidadPedir(i,c,t,col) \leq A_{c,col,t}*4+B_{c,col,t}*3 $$

In [48]:
for c in model.c:
    for t in model.t:
        for col in model.col:
            lhs=sum(model.CantidadPedir[i,c,t,col] for i in model.i)
            rhs=(model.A[c,col,t]*4+model.B[c,col,t]*3)
            model.RCajasTotales.add(lhs<=rhs)

In [49]:
model.RCajasTipoA=pe.ConstraintList()

$$ \sum_i CantidadPedir(i,c,t,col)_{tipo a}\leq A_{c,col,t}*4 $$

In [50]:
for c in model.c:
    for t in model.t:
        for col in model.col:
            lhs=sum(model.CantidadPedir[i,c,t,col]*float(int(model.LargoCont[i])==int(1600)) for i in model.i)
            rhs=model.A[c,col,t]*4
            model.RCajasTipoA.add(lhs<=rhs)

$$ \sum_i CantidadPedir(i,c,t,col)_{tipo b}\leq B_{c,col,t}*3 $$

In [51]:
model.RCajasTipoB=pe.ConstraintList()

In [52]:
for c in model.c:
    for t in model.t:
        for col in model.col:
            lhs=sum(model.CantidadPedir[i,c,t,col]*float(int(model.LargoCont[i])==int(1000)) for i in model.i)
            rhs=model.B[c,col,t]*3
            model.RCajasTipoB.add(lhs<=rhs)

In [53]:
model.RColumnasCamion=pe.ConstraintList()
for c in model.c:
    for t in model.t:
         model.RColumnasCamion.add(model.A[c,2,t]== model.A[c,1,t])
         model.RColumnasCamion.add(model.B[c,2,t]== model.B[c,1,t])


$$\forall i,t  \quad Stock[i,t]\geq \sum_{j \in [1,2,3,4]} Demanda[i,t+j]$$

In [54]:
model.RCobertura=pe.ConstraintList()
l=list(range(25))
for i in model.i:
    for t in range(1,7):
        lhs=model.Stock[i,t]
        #rhs=(model.demanda[i,t+1]+model.demanda[i,t+2]+model.demanda[i,t+3])
        rhs=0

        model.RCobertura.add(lhs>=rhs)
        l[i]=rhs
for i in model.i:
    for t in range(7,10):
        lhs=model.Stock[i,t]
        
        #rhs=l[i]
        rhs=0

        model.RCobertura.add(lhs>=rhs)


In [55]:
model.RCVolumenminimo=pe.ConstraintList()

In [56]:
for t in model.t:
    for c in model.c:
        lhs=(sum(model.CantidadPedir[i,c,t,col] for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))*750*1200*1600+
                                     +(sum(model.CantidadPedir[i,c,t,col] for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))*1000*1000*1200)/(3000*2400*13600)
        rhs=0.98*model.Y[c,t]
        model.RCVolumenminimo.add(lhs >= rhs)

In [57]:
model.bounds=pe.ConstraintList()
for i in model.i:
    for t in model.t:
        model.bounds.add(model.Stock[i,t]>=0)


In [58]:
solvername='cbc'
solverpath_exe=r"C:\Users\jmormal\Downloads\cbc-win64\cbc.exe"

solver = po.SolverFactory(solvername,executable=solverpath_exe)
# solver.options["TimeLimit"]=60
# # solver.options["MIPFocus"] = 0
# solver.options["MIPGap"] = 0.01
solver.options["seconds"]=10*60
results = solver.solve(model,tee=True)

print(results)

Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Nov 24 2021 

command line - C:\Users\jmormal\Downloads\cbc-win64\cbc.exe -seconds 600 -printingOptions all -import C:\Users\jmormal\AppData\Local\Temp\13\tmp7a9r8f5y.pyomo.lp -stat=1 -solve -solu C:\Users\jmormal\AppData\Local\Temp\13\tmp7a9r8f5y.pyomo.soln (default strategy 1)
seconds was changed from 1e+100 to 600
Option for printingOptions changed from normal to all
Presolve is modifying 108 integer bounds and re-presolving
Presolve 450 (-847) rows, 396 (-757) columns and 1546 (-1711) elements
Statistics for presolved model
Original problem has 1134 integers (54 of which binary)
Presolved problem has 378 integers (54 of which binary)
==== 324 zero objective 3 different
324 variables have objective of 0
18 variables have objective of 0.06
54 variables have objective of 1000
==== absolute objective values 3 different
324 variables have objective of 0
18 variables have objective of 0.06
54 variables have objective of 1000
==

Cbc0010I After 44000 nodes, 11521 on tree, 21023.16 best solution, best possible 20684.393 (259.00 seconds)
Cbc0004I Integer solution of 21018.6 found after 1155929 iterations and 44487 nodes (260.88 seconds)
Cbc0010I After 45000 nodes, 11516 on tree, 21018.6 best solution, best possible 20684.393 (262.87 seconds)
Cbc0010I After 46000 nodes, 11545 on tree, 21018.6 best solution, best possible 20684.393 (267.68 seconds)
Cbc0038I Full problem 450 rows 396 columns, reduced to 241 rows 213 columns - 2 fixed gives 241, 211 - still too large
Cbc0038I Full problem 450 rows 396 columns, reduced to 241 rows 211 columns - too large
Cbc0010I After 47000 nodes, 11807 on tree, 21018.6 best solution, best possible 20684.393 (271.14 seconds)
Cbc0010I After 48000 nodes, 12024 on tree, 21018.6 best solution, best possible 20684.393 (273.60 seconds)
Cbc0010I After 49000 nodes, 11824 on tree, 21018.6 best solution, best possible 20684.393 (276.81 seconds)
Cbc0010I After 50000 nodes, 11853 on tree, 21018.

Cbc0010I After 113000 nodes, 20333 on tree, 21018.6 best solution, best possible 20684.396 (540.45 seconds)
Cbc0010I After 114000 nodes, 20788 on tree, 21018.6 best solution, best possible 20684.397 (550.49 seconds)
Cbc0010I After 115000 nodes, 21031 on tree, 21018.6 best solution, best possible 20684.397 (554.16 seconds)
Cbc0010I After 116000 nodes, 21264 on tree, 21018.6 best solution, best possible 20684.397 (558.68 seconds)
Cbc0010I After 117000 nodes, 21241 on tree, 21018.6 best solution, best possible 20684.397 (564.37 seconds)
Cbc0010I After 118000 nodes, 21682 on tree, 21018.6 best solution, best possible 20684.398 (572.89 seconds)
Cbc0010I After 119000 nodes, 21960 on tree, 21018.6 best solution, best possible 20684.398 (578.42 seconds)
Cbc0010I After 120000 nodes, 22148 on tree, 21018.6 best solution, best possible 20684.398 (582.78 seconds)
Cbc0010I After 121000 nodes, 22101 on tree, 21018.6 best solution, best possible 20684.398 (589.49 seconds)
Cbc0020I Exiting on maximum 

In [59]:
for t in range(1,6):
    print(t,model.Stock[i,t].value,-model.demanda[i,t+1]-model.demanda[i,t+2]-model.demanda[i,t+3] ,
          model.Stock[i,t].value-model.demanda[i,t+1]-model.demanda[i,t+2]-model.demanda[i,t+3])
    a=-model.demanda[i,t+1]-model.demanda[i,t+2]-model.demanda[i,t+3]-model.demanda[i,t+4]
for t in range(6,10):
    print(t,model.Stock[i,t].value,a ,model.Stock[i,t].value+a)
def mean(x):
    return sum(x)/len(x)

1 6.0 -498 -492.0
2 21.0 -525 -504.0
3 33.0 -444 -411.0
4 24.0 -444 -420.0
5 42.0 -444 -402.0
6 33.0 -570 -537.0
7 54.0 -570 -516.0
8 0.0 -570 -570.0
9 16.0 -570 -554.0


In [60]:
for t in model.t:
    for c in model.c:
        print(t,c,model.Y[c,t].value)

1 1 1.0
1 2 0.0
1 3 1.0
1 4 0.0
1 5 0.0
1 6 0.0
2 1 0.0
2 2 1.0
2 3 1.0
2 4 0.0
2 5 0.0
2 6 1.0
3 1 1.0
3 2 1.0
3 3 0.0
3 4 0.0
3 5 1.0
3 6 0.0
4 1 0.0
4 2 1.0
4 3 1.0
4 4 0.0
4 5 0.0
4 6 0.0
5 1 1.0
5 2 0.0
5 3 0.0
5 4 1.0
5 5 0.0
5 6 1.0
6 1 1.0
6 2 0.0
6 3 0.0
6 4 0.0
6 5 0.0
6 6 1.0
7 1 1.0
7 2 1.0
7 3 0.0
7 4 0.0
7 5 0.0
7 6 0.0
8 1 1.0
8 2 0.0
8 3 1.0
8 4 0.0
8 5 0.0
8 6 0.0
9 1 1.0
9 2 0.0
9 3 1.0
9 4 0.0
9 5 0.0
9 6 0.0


In [61]:
utilizacion=0
volumen_por_camion=[]
for t in model.t:
    for c in model.c:
        if model.Y[c,t].value==1:
            volumen_por_camion.append(((sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))*750*1200*1600+
                                     +(sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))*1000*1000*1200)/
                                     (3000*2400*13600)))
print(volumen_por_camion)
print(mean(volumen_por_camion))
print(len(volumen_por_camion))

[1.0, 1.0, 1.0, 1.0, 0.9852941176470589, 0.9852941176470589, 0.9852941176470589, 0.9852941176470589, 0.9852941176470589, 0.9852941176470589, 1.0, 1.0, 1.0, 1.0, 0.9852941176470589, 1.0, 1.0, 1.0, 1.0, 0.9877450980392157, 0.9877450980392157]
0.9939309056956115
21


In [62]:
for t in model.t:
    print(sum(model.Y[c,t].value for c in model.c))

2.0
3.0
3.0
2.0
3.0
2.0
2.0
2.0
2.0


In [63]:
l=[]
for t in model.t:
    for i in model.i:
        1
        # print(t,i,model.Stock[i,t].value, sum(model.Y[c,t].value  for c in model.c if model.CantidadPedir[i,c,t,col].value>0))

            # if model.Y[c,t].value==1:
            #     print(t,model.A[c,col,t].value,model.B[c,col,t].value,((sum(model.CantidadPedir[i,c,t,col].value/model.PiezasCont[i] for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))*750*1200*1600+
            #                          +(sum(model.CantidadPedir[i,c,t,col].value/model.PiezasCont[i] for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))*1000*1000*1200)/
            #                          (3000*2400*13600)))

In [64]:
l=[]
for t in model.t:
    for c in model.c:
        for col in model.col:

            if model.Y[c,t].value==1:
                l.append([c,col,t,model.A[c,col,t].value,model.B[c,col,t].value,((sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))*750*1200*1600+
                                     +(sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))*1000*1000*1200)/
                                     (3000*2400*13600),[model.CantidadPedir[i,c,t,col].value for i in model.i])])
                # print([model.CantidadPedir[i,c,t,col].value for i in model.i])
                # print(t,model.A[c,col,t].value,model.B[c,col,t].value,((sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))*750*1200*1600+
                #                      +(sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))*1000*1000*1200)/
                #                      (3000*2400*13600)),
                #     sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600))
                #       ,sum(model.CantidadPedir[i,c,t,col].value for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))

In [65]:
df1=pd.DataFrame(l,columns=["c", "col", "t", "A", "B", "Volumen",])
df1

,c,col,t,A,B,Volumen
0,1,1,1,1.0,12.0,"(1.0, [4.0, 36.0])"
1,1,2,1,1.0,12.0,"(1.0, [4.0, 36.0])"
2,3,1,1,1.0,12.0,"(1.0, [4.0, 36.0])"
3,3,2,1,1.0,12.0,"(1.0, [4.0, 36.0])"
4,2,1,2,1.0,12.0,"(1.0, [4.0, 36.0])"
5,2,2,2,1.0,12.0,"(1.0, [4.0, 36.0])"
6,3,1,2,1.0,12.0,"(1.0, [4.0, 36.0])"
7,3,2,2,1.0,12.0,"(1.0, [4.0, 36.0])"
8,6,1,2,4.0,7.0,"(0.9852941176470589, [16.0, 21.0])"
9,6,2,2,4.0,7.0,"(0.9852941176470589, [16.0, 21.0])"


In [66]:
for t in model.t:
    for c in model.c:
        for i in model.i:
            print(model.CantidadPedir[i,c,t,col].value)#*float(int(model.LargoCont[i])==int(1000)))

4.0
36.0
0.0
0.0
4.0
36.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
4.0
36.0
4.0
36.0
0.0
0.0
0.0
0.0
16.0
21.0
3.0
36.0
3.0
36.0
0.0
0.0
0.0
0.0
4.0
36.0
0.0
0.0
0.0
0.0
3.0
36.0
16.0
21.0
0.0
0.0
0.0
0.0
0.0
0.0
4.0
36.0
0.0
0.0
0.0
0.0
4.0
36.0
0.0
0.0
4.0
36.0
4.0
36.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.0
21.0
4.0
36.0
4.0
36.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
4.0
36.0
0.0
0.0
4.0
36.0
0.0
0.0
0.0
0.0
0.0
0.0
4.0
35.0
0.0
0.0
4.0
36.0
0.0
0.0
0.0
0.0
0.0
0.0


In [67]:
l1=[]
a=list(range(25))
for t in range(1,7):
    for i in model.i:
            l1.append([t,i,model.Stock[i,t].value,-model.demanda[i,t+1]-model.demanda[i,t+2]-model.demanda[i,t+3] ,
          model.Stock[i,t].value-model.demanda[i,t+1]-model.demanda[i,t+2]-model.demanda[i,t+3]])
            a[i]=-model.demanda[i,t+1]-model.demanda[i,t+2]-model.demanda[i,t+3]
for t in range(7,9):
    for i in model.i:
            l1.append([t,i,model.Stock[i,t].value,a[i] ,
          model.Stock[i,t].value+a[i]])

In [68]:
df2=pd.DataFrame(l1,columns=["t","id","Stock","Demanda Proximos Dias","Diferencia Demanda"])
pd.set_option('display.max_rows', None)
df2

,t,id,Stock,Demanda Proximos Dias,Diferencia Demanda
0,1,2,0.0,-92,-92.0
1,1,1,6.0,-498,-492.0
2,2,2,0.0,-72,-72.0
3,2,1,21.0,-525,-504.0
4,3,2,1.0,-76,-75.0
5,3,1,33.0,-444,-411.0
6,4,2,16.0,-76,-60.0
7,4,1,24.0,-444,-420.0
8,5,2,12.0,-80,-68.0
9,5,1,42.0,-444,-402.0


In [69]:
# for t in model.t:
#     print(t,sum(model.CantidadPedir[i,c,t,col].value for c in model.c for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))
print(sum(model.CantidadPedir[i,c,t,col].value for t in model.t for c in model.c for i in model.i for col in model.col if int(model.LargoCont[i])==int(1600)))
print(sum(model.CantidadPedir[i,c,t,col].value for t in model.t for c in model.c for i in model.i for col in model.col if int(model.LargoCont[i])==int(1000)))

236.0
1420.0


In [70]:
df_demanda

,Magnum,A-cont
Collection in supplier,,
day 1,138,16
day 2,171,48
day 3,204,20
day 4,123,24
day 5,198,28
day 6,123,24
day 7,123,24
day 8,198,32
day 9,126,20


In [71]:
CamionesDia=[]
for t in model.t:
    CamionesDia.append(sum(model.Y[c,t].value for c in model.c))
df_CamionesDia=pd.DataFrame(CamionesDia,columns=["CamionesDia"])
df_CamionesDia.to_csv("CamionesDia.csv")

In [72]:
stock=[]
for t in model.t:
    for i in model.i:
        stock.append([t,i,model.Stock[i,t].value])
df_stock=pd.DataFrame(stock,columns=["t","id","Stock"])
df_stock.to_csv("Stock.csv")

In [73]:
df2.loc[df2["id"]==1]["t"]

1     1
3     2
5     3
7     4
9     5
11    6
13    7
15    8
Name: t, dtype: int64

In [74]:
cantidadpedir_pro=[]
for t in model.t:
    pro=[]
    for i in model.i:
        pro.append(sum(model.CantidadPedir[i,c,t,col].value  for c in model.c for col in model.col))
    cantidadpedir_pro.append(pro)
df_cantidadpedir_pro=pd.DataFrame(cantidadpedir_pro,columns=[str(i) for i in model.i])

In [75]:
df_cantidadpedir_pro.to_csv("CantidadPedir.csv")

In [76]:
cantidadpedir_acu=[]
for t in model.t:
    pro=[]
    for i in model.i:
        pro.append(sum(model.CantidadPedir[i,c,t1,col].value  for c in model.c for col in model.col for t1 in list(model.t)[0:t]))
    cantidadpedir_acu.append(pro)

In [77]:
df_cantidadpedir_acu=pd.DataFrame(cantidadpedir_acu, columns=[str(i) for i in model.i])
df_cantidadpedir_acu["2"]
df_cantidadpedir_acu.to_csv("CantidadPedirAcumulado.csv")